In [3]:
#!/usr/bin/env python
from parallel import DDCASPT2
import pickle, os, shutil
from glob import glob
import numpy as np
from joblib import Parallel, delayed
import pandas as pd
from math import sin, cos, pi
from tqdm.notebook import tqdm
# Geom manipulate
from AaronTools.geometry import Geometry
from tqdm import tqdm
from time import perf_counter




In [5]:
# Take [Fe$^{IV}$(O)(H$_2$O)$_4$]$^{+2}$ from Phys. Chem. Chem. Phys., 2018,20, 28786-28795
radius_range=np.linspace(1,3,100)



In [ ]:




# In[ ]:


def gen_run(r,path):
    with open(os.path.join(path,f'{r}_run.sh'),'w') as g:
        g.write(f"""#!/bin/bash
#This file is a submission script to request the ISAAC resources from Slurm
#SBATCH --account=ACF-UTK0022             # The project account to be charged
#SBATCH --job-name=features_ironoxo_{r}		       #The name of the job
#SBATCH --nodes=1                     # Number of nodes
#SBATCH --ntasks-per-node=48          # cpus per node
#SBATCH --partition=condo-kvogiatz            # If not specified then default is "campus"
#SBATCH --qos=condo-kvogiatz
#SBATCH --time=0-08:00:00             # Wall time (days-hh:mm:ss)
#SBATCH --error=job.e%J	       # The file where run time errors will be dumped
#SBATCH --output=job.o%J	       # The file where the output of the terminal will be dumped



# PATCH!
# MOVE TO WORKING DIR
echo $SLURM_SUBMIT_DIR
cd $SLURM_SUBMIT_DIR
pwd
# DEFINE WorkDir
mkdir $SLURM_SUBMIT_DIR/$SLURM_JOBID
export WorkDir=$SLURM_SUBMIT_DIR/$SLURM_JOBID
export MOLCAS_WORKDIR="/lustre/isaac/scratch/gjones39"
export MOLCAS="/lustre/isaac/proj/UTK0022/GMJ/Test/build"
echo $WorkDir



module purge
module load intel-compilers/2021.2.0
module load hdf5/1.10.8-intel
module load cmake/3.23.2-intel

# DEFINE WorkDir
echo "Starting at $(date)"
echo "Running on hosts: $SLURM_NODELIST"
echo "Running on $SLURM_NNODES nodes."
echo "Running $SLURM_NTASKS tasks."
echo "Current working directory is $(pwd)"

# THE COMMAND

conda activate /lustre/isaac/proj/UTK0022/GMJ/modules/anaconda3/envs/ddcaspt2
python3 {r}_newddcaspt2.py >> {r}_genfeat.out 2>&1 
# CLEAN-UP AND EXIT
rm -r $SLURM_SUBMIT_DIR/$SLURM_JOBID
echo "Program finished with exit code $? at: $(date)"
""")    




In [ ]:
for idxr, r in tqdm(enumerate(radius_range)):
# Create radius subdirectory
    radstr = f"{r:.2f}"
    print("\nRADIUS:",radstr)
    rad_dir = os.path.join(os.getcwd(),radstr)
    pythoninput = f"""#!/usr/bin/env python
from parallel import DDCASPT2
import pickle, os, shutil
from glob import glob
import numpy as np
from joblib import Parallel, delayed
import pandas as pd
from math import sin, cos, pi
from tqdm.notebook import tqdm
# Geom manipulate
from AaronTools.geometry import Geometry
from tqdm import tqdm
from time import perf_counter


d = DDCASPT2('{rad_dir}','ANO-RCC-VDZP',{radstr},10,8,None,previous=None,symmetry=1,spin=4,UHF=True,charge=2,clean=False,n_jobs=48)(run=False)
"""
    with open(f"{radstr}_newddcaspt2.py",'w') as f:
        f.write(pythoninput)
    gen_run(radstr,os.getcwd())